In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set(style="darkgrid")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [ ]:
data = spark.read.csv('../input/customer-churn/customer_churn.csv',inferSchema=True, header=True)

In [ ]:
data.printSchema()

In [ ]:
data.describe().show()

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
final_data = output.select('features','churn')

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol='churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

**Using AUC**

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
print(auc)

**Predict on Brand New Un-Labeled Data**

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
new_customers = spark.read.csv('../input/customers/new_customers.csv',inferSchema=True, header=True)

In [ ]:
new_customers.printSchema()

In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

In [ ]:
final_results = final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()